In [ ]:
!pip install great_expectations==0.18.19

In [ ]:
import great_expectations as gx
import pandas as pd

context = gx.get_context()

# Carregando os Dados
df = pd.read_csv('olist_order_items_dataset.csv')

#Convertendo para um Dataset do GX
validator = context.sources.pandas_default.read_dataframe(df)

print("Dados carregados com sucesso! Linhas:", len(df))

In [ ]:
# DEFINIÇÃO DE REGRAS

# Regra de Completude: IDs não podem ser nulos
validator.expect_column_values_to_not_be_null("order_id")
validator.expect_column_values_to_not_be_null("product_id")

# Regra de Validade: Preço e Frete não podem ser negativos
validator.expect_column_values_to_be_between("price", min_value=0)
validator.expect_column_values_to_be_between("freight_value", min_value=0)

# Regra de Tipo: O ID do vendedor deve ser texto (string)
validator.expect_column_values_to_be_of_type("seller_id", "str")

validator.save_expectation_suite()
print("Regras de Data Quality definidas com sucesso!")

In [ ]:
import json

# Cria um Checkpoint
checkpoint = context.add_or_update_checkpoint(
    name="checkpoint_olist",
    validator=validator,
)

# Validação
results = checkpoint.run()

print(f"\nStatus da Validação: {'SUCESSO ✅' if results['success'] else 'FALHA ❌'}")

stats = results.list_validation_results()[0]["statistics"]

print("\n-------------------------------------------------------")
print(json.dumps(stats, indent=4))
print("----------------------------------------------------")

In [ ]:
# COMMON DATA MODEL

df_cdm = df.rename(columns={
    "order_id": "SalesOrderNumber",
    "order_item_id": "LineNumber",
    "product_id": "ProductNumber",
    "seller_id": "PartyId",
    "price": "TransactionAmount",
    "freight_value": "ShippingAmount",
    "shipping_limit_date": "RequestedDeliveryDate"
})

df_cdm.to_csv("GOLD_SALES_ORDER_CDM.csv", index=False)

print("Arquivo GOLD_SALES_ORDER_CDM.csv gerado com sucesso!")
print("Arquivos salvos em files")